### Generate text description

In [12]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from openai_utils.openai import client
from transformers import AutoTokenizer, AutoModel



/home/wenhao/.conda/envs/polytexter/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
path = "/home/wenhao/Project/greatxue/polyTexter/proj/datasets/comprehensive_tg_raw.csv"
df_raw = pd.read_csv(path)
df = df_raw[['SMILES', 'tg']]
df

,SMILES,tg
0,*C1COC2C1OCC2Oc1ccc(cc1)CNC(=O)CCCCCCC(=O)NCc1...,21.581731
1,*OC(CCC(OC(=O)Nc1ccc(cc1)Cc1ccc(cc1)NC(=O)*)C)C,63.589338
2,*OC(=O)c1ccc(cc1)C(=O)OCCCC(=O)NCc1ccc(cc1)CNC...,53.557261
3,*OC(=O)NCCNC(=O)OCC*,5.896093
4,*SCCCCC*,-55.378610
...,...,...
561,*NNC(=O)CCC(=O)NNC(=O)CCCCCCCCC(=O)*,64.698504
562,*NC(=O)CCC(=O)NCCCCCCCC*,69.221302
563,*NC(C(C(=O)*)(C)C)c1ccccc1,154.359507
564,*OC(=O)C/C=C/CC(=O)OCCCCCCCCCCCCCC*,-41.101589


In [13]:
df.to_csv("/home/wenhao/Project/greatxue/polyTexter/proj/datasets/tg_raw.csv", index=False)

In [3]:
def ques_gpt(prompt="Hi, how do you do today?"):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
    )
    if response.choices[0]:
        return response.choices[0].message.content.strip()
    else:
        print('No valid GPT-response generated.')

In [ ]:
responses = []

for index, row in df.iterrows():
    smiles = row['SMILES']
    print(f"{index} Processing SMILES: {smiles}")
    gpt_response = ques_gpt(f"From a structural perspective, please provide some inferential information related to the glass transition temperature of the polymer represented by the SMILES {smiles}.")
    #print(f"{index} GPT Response: {gpt_response}\n")
    responses.append(gpt_response)

prompt_df = pd.DataFrame(responses, columns=['tg_description'])
prompt_df.to_csv("/home/wenhao/Project/greatxue/polyTexter/proj/datasets/tg_comprehensive_description.csv", index=True)
print("The description has been saved.")


0 Processing SMILES: *C1COC2C1OCC2Oc1ccc(cc1)CNC(=O)CCCCCCC(=O)NCc1ccc(cc1)O*


1 Processing SMILES: *OC(CCC(OC(=O)Nc1ccc(cc1)Cc1ccc(cc1)NC(=O)*)C)C
2 Processing SMILES: *OC(=O)c1ccc(cc1)C(=O)OCCCC(=O)NCc1ccc(cc1)CNC(=O)CCC*
3 Processing SMILES: *OC(=O)NCCNC(=O)OCC*
4 Processing SMILES: *SCCCCC*
5 Processing SMILES: *Oc1ccc(cc1)C(=O)OC(=O)c1ccc(cc1)OCCCCCC*
6 Processing SMILES: *c1[nH]c(cc1CC(=O)OCCCCCCCC)*
7 Processing SMILES: *C(C*)(CC(=O)OCCCCCCCCCC)C(=O)OCCCCCCCCCC
8 Processing SMILES: *OCC1C(C1)C*
9 Processing SMILES: *N(C(=O)CCCCCCCCCCCCCCCCC(=O)N(CCCCCC*)C)C
10 Processing SMILES: *O[Si](*)(CCC(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F)C
11 Processing SMILES: *S(=O)(=O)c1ccc(cc1)C(=O)CNc1ccc(cc1)NCC(=O)c1ccc(cc1)*
12 Processing SMILES: *S(=O)(=O)c1ccc(cc1)C(=O)NCCNC(=O)c1ccc(cc1)*
13 Processing SMILES: *c1cc2c(C(=O)N(C2=O)c2ccc(cc2)C(c2ccc(cc2)N2C(=O)c3c(C2=O)cc(cc3)C(=O)*)CCCCC)cc1
14 Processing SMILES: *NC(=O)C(=O)NCCCCCCCCCCNC(=O)CCCCCCCCC(=O)NCCCCCCCCCC*
15 Processing SMILES: *OC(=O)c1ccc(cc1)C(=O)OCCCCCCCCCCCCCCCCCCCC*
16 Processing SMILES: *Oc1ccc(cc1)

In [ ]:
path = "/home/wenhao/Project/greatxue/polyTexter/proj/datasets/tg_comprehensive_description.csv"
df_text = pd.read_csv(path)

df_text

,Unnamed: 0,tg_description
0,0,To infer the glass transition temperature (\( ...
1,1,The SMILES string provided represents a polyme...
2,2,To infer the glass transition temperature (\( ...
3,3,The polymer represented by the SMILES *OC(=O)N...
4,4,The polymer represented by the SMILES *SCCCCC*...
...,...,...
561,561,The SMILES string you provided describes a pol...
562,562,To infer the glass transition temperature (Tg)...
563,563,"The provided SMILES notation, *NC(C(C(=O)*)(C)..."
564,564,The given SMILES notation represents a polymer...


### Generate tokenized descriptions

In [5]:
df_merged = pd.concat([df, df_text[['tg_description']]], axis=1)
df_merged.to_csv('smiles_with_description_tg.csv', index=True)

In [6]:
################################### TEST ##########################################
model_name = "meta-llama/Meta-Llama-3.1-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

short = "This is tokenizer."
long = "This is a long long long long tokenizer."
tokens = tokenizer.encode(long, add_special_tokens=True, return_tensors="pt")

print("Original text:", long)
print("Tokens:", tokens)
print("Decoded text:", tokenizer.decode(tokens[0])) 
with torch.no_grad():
    outputs = model(input_ids=tokens) 
    token_embeddings = outputs.last_hidden_state 

print("Token Embeddings shape:", token_embeddings.shape)

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.09s/it]


Original text: This is a long long long long tokenizer.
Tokens: tensor([[128000,   2028,    374,    264,   1317,   1317,   1317,   1317,  47058,
             13]])
Decoded text: <|begin_of_text|>This is a long long long long tokenizer.
Token Embeddings shape: torch.Size([1, 10, 4096])


In [8]:
model_name = "meta-llama/Meta-Llama-3.1-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

input_csv_path = '/home/wenhao/Project/greatxue/polyTexter/proj/datasets/tg_description.csv'
df = pd.read_csv(input_csv_path)

if 'tg_description' not in df.columns:
    raise ValueError("No tg_description found")

embeddings_list = []
index = 0

for description in df['tg_description']:
    tokens = tokenizer.encode(description, add_special_tokens=True, return_tensors="pt")

    with torch.no_grad():
        outputs = model(input_ids=tokens)
        token_embeddings = outputs.last_hidden_state # (1, sequence_length, hidden_size)

    mean_embedding = token_embeddings.mean(dim=1).squeeze().numpy() # avg the sequence_length
    embeddings_list.append(mean_embedding)
    print(f"The {index} embedding appended.")
    print(f"Shape: {mean_embedding.shape}")
    index += 1

embeddings_array = np.array(embeddings_list)

output_npy_path = 'tokenized_embeddings.npy'
np.save(output_npy_path, embeddings_array)
print("===================The npy file is saved.===================")

df_embeddings = pd.DataFrame(embeddings_array)
output_csv_path = 'tokenized_embeddings.csv'
df_embeddings.to_csv(output_csv_path, index=False, header=False)
print("===================The csv file is saved.===================")

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.02s/it]


The 0 embedding appended.
Shape: (4096,)
The 1 embedding appended.
Shape: (4096,)
The 2 embedding appended.
Shape: (4096,)
The 3 embedding appended.
Shape: (4096,)
The 4 embedding appended.
Shape: (4096,)
The 5 embedding appended.
Shape: (4096,)
The 6 embedding appended.
Shape: (4096,)
The 7 embedding appended.
Shape: (4096,)
The 8 embedding appended.
Shape: (4096,)
The 9 embedding appended.
Shape: (4096,)
The 10 embedding appended.
Shape: (4096,)
The 11 embedding appended.
Shape: (4096,)
The 12 embedding appended.
Shape: (4096,)
The 13 embedding appended.
Shape: (4096,)
The 14 embedding appended.
Shape: (4096,)
The 15 embedding appended.
Shape: (4096,)
The 16 embedding appended.
Shape: (4096,)
The 17 embedding appended.
Shape: (4096,)
The 18 embedding appended.
Shape: (4096,)
The 19 embedding appended.
Shape: (4096,)
The 20 embedding appended.
Shape: (4096,)
The 21 embedding appended.
Shape: (4096,)
The 22 embedding appended.
Shape: (4096,)
The 23 embedding appended.
Shape: (4096,)
Th

### Check `.npy`

In [1]:
import numpy as np

loaded_feature = np.load('/home/wenhao/Project/greatxue/polyTexter/proj/lfs/token_embedding/tokenized_embeddings.npy', allow_pickle=False)
print("Loaded feature type:", type(loaded_feature))

if isinstance(loaded_feature, np.ndarray):
    features = loaded_feature[0]  
else:
    raise ValueError("Loaded feature is not a numpy array")

print("Shape of loaded feature:", loaded_feature.shape)
print("Contents of loaded feature:", loaded_feature)

Loaded feature type: <class 'numpy.ndarray'>
Shape of loaded feature: (566, 4096)
Contents of loaded feature: [[-0.7884431   0.33653435  0.78608364 ...  0.616118    0.81767315
   0.7376049 ]
 [-1.1838228   0.00815376 -0.00630396 ...  0.807461    1.0933573
   0.64416426]
 [-0.84809536  0.49608606  0.59324306 ...  0.6211122   0.89937
   0.48507413]
 ...
 [-0.807832    0.8032247   0.41008565 ...  0.82063425  1.0088054
   0.09457733]
 [-0.6035069   0.63234764  0.07720878 ...  0.55987245  1.1149347
   1.2357465 ]
 [-1.231941    0.54752594  0.884722   ...  0.95684725  0.6428626
   0.5924669 ]]


In [2]:
#mean = np.mean(loaded_feature, axis=0, keepdims=True)
#std = np.std(loaded_feature, axis=0, keepdims=True)
#l2_normalized_embeddings = (loaded_feature - mean) / std

l2_normalized_embeddings = (loaded_feature - np.min(loaded_feature)) / (np.max(loaded_feature) - np.min(loaded_feature))
print("Shape of loaded feature:", l2_normalized_embeddings.shape)
print("Contents of loaded feature:", l2_normalized_embeddings)
np.save('/home/wenhao/Project/greatxue/polyTexter/proj/lfs/token_embedding/tokenized_embeddings_norm.npy', l2_normalized_embeddings)


Shape of loaded feature: (566, 4096)
Contents of loaded feature: [[0.4142758  0.45580554 0.47240108 ... 0.46612665 0.47356725 0.47061148]
 [0.39967996 0.44368303 0.4431493  ... 0.47319028 0.4837444  0.467162  ]
 [0.4120737  0.46169555 0.46528217 ... 0.466311   0.4765832  0.46128902]
 ...
 [0.41356006 0.47303388 0.45852074 ... 0.47367656 0.4806231  0.44687346]
 [0.4211029  0.4667258  0.44623226 ... 0.4640503  0.484541   0.4890009 ]
 [0.39790362 0.46359453 0.47604245 ... 0.47870502 0.46711394 0.46525356]]


In [13]:
class EmbeddingProjector(nn.Module):
    def __init__(self, input_dim=4096, output_dim=128):
        super(EmbeddingProjector, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = nn.Tanh()
    def forward(self, x):
        x = self.fc(x)
        x = self.activation(x)
        return x

projector = EmbeddingProjector()

tensor_input = torch.tensor(l2_normalized_embeddings, dtype=torch.float32)
output_embeddings = projector(tensor_input) 

print(output_embeddings)
print(output_embeddings.shape)

output_array = output_embeddings.detach().cpu().numpy()
np.save('output_embedding.npy', output_array)

tensor([[ 0.0074,  0.0039, -0.0349,  ..., -0.0016, -0.0325, -0.0061],
        [ 0.0156, -0.0040, -0.0204,  ..., -0.0062, -0.0241,  0.0220],
        [ 0.0231,  0.0030, -0.0395,  ..., -0.0100, -0.0408, -0.0218],
        ...,
        [ 0.0146, -0.0117, -0.0469,  ..., -0.0097, -0.0183,  0.0026],
        [ 0.0351,  0.0018, -0.0576,  ..., -0.0069, -0.0419,  0.0076],
        [ 0.0213,  0.0118, -0.0455,  ..., -0.0110, -0.0174, -0.0205]],
       grad_fn=<TanhBackward0>)
torch.Size([566, 128])
